## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [23]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout,Activation
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


In [24]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [25]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [26]:

classifier=Sequential()
#卷積組合
classifier.add(Convolution2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())
classifier.add(Activation('relu'))
'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,(3,3),activation='relu'))
classifier.add(BatchNormalization())

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(512)) #output_dim=100,activation=relu
classifier.add(Activation('relu'))
classifier.add(Dropout(0.25))
#輸出
classifier.add(Dense(10,activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Epoch 1/100
500/500 [==============================] - 58s 115ms/step - loss: 1.6885 - accuracy: 0.4663
Epoch 2/100
500/500 [==============================] - 57s 114ms/step - loss: 0.8504 - accuracy: 0.7048
Epoch 3/100
500/500 [==============================] - 57s 113ms/step - loss: 0.5769 - accuracy: 0.8014
Epoch 4/100
500/500 [==============================] - 57s 113ms/step - loss: 0.3973 - accuracy: 0.8645
Epoch 5/100
500/500 [==============================] - 58s 117ms/step - loss: 0.2560 - accuracy: 0.9143
Epoch 6/100
500/500 [==============================] - 56s 111ms/step - loss: 0.1914 - accuracy: 0.9348
Epoch 7/100
500/500 [==============================] - 55s 110ms/step - loss: 0.1554 - accuracy: 0.9479
Epoch 8/100
500/500 [==============================] - 56s 111ms/step - loss: 0.1540 - accuracy: 0.9482
Epoch 9/100
500/500 [==============================] - 56s 112ms/step - loss: 0.1208 - accuracy: 0.9608
Epoch 10/100
500/500 [==============================] - 64s 128m

500/500 [==============================] - 67s 134ms/step - loss: 0.0211 - accuracy: 0.9943
Epoch 80/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0253 - accuracy: 0.9924
Epoch 81/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0199 - accuracy: 0.9941
Epoch 82/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0208 - accuracy: 0.9934
Epoch 83/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0230 - accuracy: 0.9931
Epoch 84/100
500/500 [==============================] - 67s 133ms/step - loss: 0.0198 - accuracy: 0.9947
Epoch 85/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0245 - accuracy: 0.9927
Epoch 86/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0230 - accuracy: 0.9933
Epoch 87/100
500/500 [==============================] - 67s 134ms/step - loss: 0.0172 - accuracy: 0.9950
Epoch 88/100
500/500 [==============================] - 67s 134ms/st

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [27]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[1.6982608e-12, 5.5414468e-13, 2.3400647e-08, 7.2092521e-01,
        2.7851602e-01, 7.3282846e-09, 2.9420742e-06, 2.0050114e-14,
        5.5588176e-04, 3.5099629e-10]], dtype=float32)